# Caracterização Climática de São Paulo

Por que interessante: São Paulo é a maior metrópole do Brasil e apresenta um clima subtropical com estações relativamente bem definidas. A cidade é excelente para estudar como a urbanização impacta o microclima, gerando efeitos como as ilhas de calor urbanas. Além disso, a proximidade da Serra do Mar traz eventos de precipitação intensa, influenciando a análise das chuvas e enchentes.
Aspectos climáticos: Verões quentes e chuvosos, invernos mais frios e secos, com maior amplitude térmica anual em comparação a cidades tropicais.

# Definir as estações meteorológicas que possam representar melhor a cidade.

o objetivo desse projeto é caracterizar os microclimas da cidade de são paulo e compará-los afim de verificar se existem diferenças significativas provocadas pelas características dos territórios. A cidade é extensa dividida em cinco regioes norte sul leste oeste cada uma com ocupações distintas.

A primeira etapa é definir quais estacoes meteorológicas de cada regiao serao utilizadas. O INMET (Instituto Nacional de Meteorologia) é a fonte dos dados desse projeto e fornece uma base de dados com as coordenadas de cada estacao no Brasil, assim como os dados coletados de cada estacao.

Para cada regiao serão selecionadas três estacoes, ou seja, sera feitqa uma triangulacao de forma que possa ser representativa.

## Escolha das estacoes

O INMET fornece dois catálogos com suas estacoes - estações automáticas e estações convencionais. Cada catalogo possui cinco dimensões: 

In [6]:
import os
import logging
import csv
import pandas as pd
import folium

sourcesPath = os.path.join(os.getcwd(), "sources")

fileCount = 0
representativeDir = None

for root, dirs, files in os.walk(sourcesPath):
    for file in files:
        if not file.startswith("INMET_SE_SP_"):
            filePath = os.path.join(root, file)
            try:
                os.remove(filePath)
            except Exception as e:
                logging.error(f"Erro ao excluir {filePath}: {e}")
    
    if not os.listdir(root):
        try:
            os.rmdir(root)
        except Exception as e:
            logging.error(f"Erro ao excluir o diretório {root}: {e}")
    
    dirFileCount = len(files)
    if dirFileCount > fileCount:
        fileCount = dirFileCount
        representativeDir = root

with open(f"weatherStations.csv", mode="w", newline="") as wS:
    writer = csv.writer(wS)
    writer.writerow(["estacao", "codigo", "latitude", "longitude"])

    for file in os.listdir(representativeDir):
        filePath = os.path.join(representativeDir, file)
        
        with open(filePath, mode="r") as fP:
            reader = csv.reader(fP, delimiter=";")
            metaData = {row[0]: row[1] for row in reader if len(row) == 2}
        
            station = metaData.get("ESTACAO:").strip()
            code = metaData.get("CODIGO (WMO):").strip()
            latitude = metaData.get("LATITUDE:").strip().replace(",", ".")
            longitude = metaData.get("LONGITUDE:").strip().replace(",", ".")

            writer.writerow([station, code, latitude, longitude])

wS = pd.read_csv("weatherStations.csv", sep=",")

map = folium.Map(location=[wS["latitude"].mean(), wS["longitude"].mean()], zoom_start=6)

for id, row in wS.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row.get("estacao", "codigo"),
        icon=folium.Icon(color="black")
    ).add_to(map)

folium.TileLayer(
    tiles="http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
    overlay=False,
    control=True
).add_to(map)

map